## Грузим задание

In [3]:
! wget -nc https://github.com/artonson/hse-stat-course-2018/raw/master/homework-handouts/homework-3.pdf

--2018-12-17 22:14:40--  https://github.com/artonson/hse-stat-course-2018/raw/master/homework-handouts/homework-3.pdf
Распознаётся github.com (github.com)… 140.82.118.4, 140.82.118.3
Подключение к github.com (github.com)|140.82.118.4|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/artonson/hse-stat-course-2018/master/homework-handouts/homework-3.pdf [переход]
--2018-12-17 22:14:40--  https://raw.githubusercontent.com/artonson/hse-stat-course-2018/master/homework-handouts/homework-3.pdf
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 151.101.244.133
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|151.101.244.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 200767 (196K) [application/octet-stream]
Сохранение в: «homework-3.pdf»

homework-3.pdf      100%[===================>] 196,06K  1,01MB/s    за 0,2s    

2018-12-17 22:14:41 (1,01 

### Грузим данные

In [2]:
! wget -nc -P data/ https://raw.githubusercontent.com/artonson/hse-stat-course-2018/master/homework-handouts/passengercarmileage.txt

--2018-12-17 22:11:06--  https://raw.githubusercontent.com/artonson/hse-stat-course-2018/master/homework-handouts/passengercarmileage.txt
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 151.101.244.133
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|151.101.244.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 2765 (2,7K) [text/plain]
Сохранение в: «data/passengercarmileage.txt»

passengercarmileage 100%[===================>]   2,70K  --.-KB/s    за 0,004s  

2018-12-17 22:11:06 (759 KB/s) - «data/passengercarmileage.txt» сохранён [2765/2765]



# Задание 1.

Заметим, что $E[X] = \lambda, D[X] = \lambda$. Введем оценку $\hat\lambda = \overline X$ -- ассимптотичски нормальная оценка.
$$
W = \dfrac{\hat\lambda - \lambda_0}{\hat{se}}
$$
$H_0$ отклоняем, если $|W| > z_{\alpha/2}$

In [136]:
import numpy as np
import scipy.stats
from tqdm import tqdm_notebook as tqdm

In [305]:
lambda_0 = 1
n = 20
alpha = 0.05

In [306]:
def h0_accept():
    X = np.random.poisson(lam=lambda_0, size=n)
    W = (X.mean() - lambda_0) / X.std()
    #print(W)
    z_alpha2 = scipy.stats.norm.interval(alpha)[1]
    #print(z_alpha2)
    return not(np.abs(W) > z_alpha2)

In [307]:
n_exp = 10**4
count_accepted = 0
for i in tqdm(range(n_exp)):
    count_accepted += h0_accept()
print('Доля экспериментов с отклоненной H0 = %f' % (1 - count_accepted / n_exp))

Доля экспериментов с отклоненной H0 = 0.762200


In [308]:
p = scipy.stats.norm.cdf(scipy.stats.norm.interval(alpha)[1])
p = 2 * p - 1
print( p )

0.050000000000000044
